<a href="https://colab.research.google.com/github/caua-sathler/NEURAL-NETWORKS/blob/main/FL-PSO-SGD/FL_PSO_SGD_Iris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pega o erro de cada um e compara. A partícula com menor erro pede o gbest.

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, Subset, TensorDataset
from torchvision import datasets, transforms
import torch.nn.functional as F
import copy
import random
import csv
import torchvision.models as models
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [15]:
#Hiperparâmetros
BATCH_SIZE = 64
EPOCHS_PER_CLIENT = 10
NUM_CLIENTS = 10
NUM_ROUNDS = 100
INERTIA, C1, C2 = 0.9, 0.8, 0.9
# LEARNING_RATE = 0.001

# Configuração do dispositivo
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'training on {DEVICE}')

training on cuda


In [22]:
#Seeds para reprodutibilidade
random.seed(123)
torch.manual_seed(123)
torch.cuda.manual_seed(123)

# # Função para inicializar o modelo de acordo com a arquitetura
# def init_model(architecture="squeezenet"):
#     if architecture == "squeezenet":
#         #model = models.squeezenet1_1(weights=None)
#         model = models.squeezenet1_1(weights=models.SqueezeNet1_1_Weights.IMAGENET1K_V1)
#         model.classifier[1] = nn.Conv2d(512, 10, kernel_size=(1, 1), stride=(1, 1))
#     elif architecture == "mobilenet_v2":
#         #model = models.mobilenet_v2(weights=None)
#         model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1)
#         model.classifier[1] = nn.Linear(model.classifier[1].in_features, 10)
#     elif architecture == "resnet18":
#         #model = models.resnet18(weights=None)
#         model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
#         model.fc = nn.Linear(model.fc.in_features, 10)
#     elif architecture == "alexnet":
#         #model = models.alexnet(weights=None)
#         model = models.alexnet(weights=models.AlexNet_Weights.IMAGENET1K_V1)
#         model.classifier[6] = nn.Linear(model.classifier[6].in_features, 10)
#     else:
#         raise ValueError("Unsupported architecture. Choose from 'squeezenet', 'mobilenet_v2', 'resnet18', 'alexnet'.")
#     return model.to(DEVICE)

# Função para criar um subconjunto aleatório de um dataset
def create_subset(dataset, subset_size):
    indices = list(range(len(dataset)))
    subset_indices = random.sample(indices, subset_size)
    return Subset(dataset, subset_indices)

# # Função para salvar os resultados em um arquivo CSV
# def write_csv(algorithm_name, architecture, results):
#     file_name = f'{algorithm_name}_{architecture}_PreTreinado_CIFAR10_output.csv'
#     with open(file_name, 'w', encoding='utf-8', newline='') as f:
#         writer = csv.writer(f)
#         writer.writerow(["Epoch", "Test Loss", "Test Accuracy"])
#         for i, result in enumerate(results):
#             writer.writerow([i + 1] + result)


class MLP(nn.Module): # Class MLP heritage from Module
  def __init__(self, input_dim, output_dim): # Constructor. 'self' in Python is similar to 'this' in C++.
    super(MLP, self).__init__() # Call Module constructor first to guarantee initialization
    # Defining each layer: type and number of neurons
    self.fc1 = nn.Linear(input_dim, 16)
    self.fc2 = nn.Linear(16, 8)
    self.fc3 = nn.Linear(8, 8)
    self.fc4 = nn.Linear(8, output_dim)

    # Defining each activation function
  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))
    x = self.fc4(x) # No activation function
    return x

class Particle:
    def __init__(self, particle_id, model, data):
        self.particle_id = particle_id
        self.model = model.to(DEVICE)
        self.data = data  # `self.data` agora armazena o dataset, não o DataLoader
        self.local_best_model = copy.deepcopy(model.state_dict())
        self.local_best_score = float('inf')
        self.velocity = {name: torch.zeros_like(param) for name, param in model.named_parameters()}
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.007, weight_decay=1e-5)

    def train_particle_adam(self, criterion):
        self.model.train()
        train_loader = DataLoader(self.data, batch_size=BATCH_SIZE, shuffle=True)
        for _ in range(EPOCHS_PER_CLIENT):
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                self.optimizer.zero_grad()
                outputs = self.model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()

        val_loss = self.evaluate_loss(criterion)
        if val_loss < self.local_best_score:
            self.local_best_score = val_loss
            self.local_best_model = copy.deepcopy(self.model.state_dict())
        return self.particle_id, val_loss

    def evaluate_loss(self, criterion):
        self.model.eval()
        total_loss = 0
        val_loader = DataLoader(self.data, batch_size=BATCH_SIZE, shuffle=False)
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                outputs = self.model(inputs)
                loss = criterion(outputs, labels)
                total_loss += loss.item()
        return total_loss / len(val_loader)

    def train_particle_pso(self, global_best_model, INERTIA, C1, C2):
        for name, param in self.model.named_parameters():
            if param.grad is None:
                continue
            local_rand = random.random()
            global_rand = random.random()
            self.velocity[name] = (
                INERTIA * self.velocity[name]
                + C1 * local_rand * (self.local_best_model[name].to(DEVICE) - param.data)
                + C2 * global_rand * (global_best_model[name].to(DEVICE) - param.data)
            )
            param.data += self.velocity[name]
            # param.data = param.data - LEARNING_RATE * param.grad

def train_federated():
    # transform = transforms.Compose([
    #     transforms.Resize((224, 224)),
    #     transforms.ToTensor(),
    #     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    # ])

    # trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    # trainloaders = [create_subset(trainset, 5000) for _ in range(NUM_CLIENTS)]

    # testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
    # testloader = DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False)

    iris = datasets.load_iris() # Load dataset Iris. Classification. 150 instances, 4 features and 3 classes.
    X = iris.data # X receives all features of each flower
    y = iris.target # y receives all classes of each flower

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # Spliting data: 20% to test and 80% to training

    # Normalizing data
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Transforming data in tensor
    X_train = torch.tensor(X_train, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.long)
    X_test = torch.tensor(X_test, dtype=torch.float32)
    y_test = torch.tensor(y_test, dtype=torch.long)

        # Criando datasets
    trainset = TensorDataset(X_train, y_train)
    testset = TensorDataset(X_test, y_test)

    # Criando subconjuntos para cada cliente
    SUBSET_SIZE = 12
    trainloaders = [create_subset(trainset, SUBSET_SIZE) for _ in range(NUM_CLIENTS)]

    # Criando DataLoader para o conjunto de teste
    testloader = DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False)

    input_dim = X_train.shape[1]  # Número de atributos (4 para o Iris)
    output_dim = len(torch.unique(y_train))  # Número de classes (3 para o Iris)

    server_model = MLP(input_dim, output_dim).to(DEVICE)
    particles = [Particle(i, copy.deepcopy(server_model), trainloaders[i]) for i in range(NUM_CLIENTS)]
    criterion = nn.CrossEntropyLoss()

    global_best_score = float('inf')
    global_best_model = copy.deepcopy(server_model.state_dict())
    server_evaluate_acc = []

    for epoch in range(NUM_ROUNDS):
        server_result = []
        for particle in particles:
            if epoch > 0:
                particle.train_particle_pso(global_best_model, INERTIA, C1, C2)
            pid, val_loss = particle.train_particle_adam(criterion)
            server_result.append((pid, val_loss))

        # print(server_result)
        sorted_result = sorted(server_result, key=lambda x: x[1])
        # print(sorted_result)
        top_3_particles = sorted_result[:3]
        # print(top_3_particles)

        # best_particle = min(server_result, key=lambda x: x[1])
        best_particle = random.choice(top_3_particles)
        best_particle_id = best_particle[0]  # ID da melhor partícula
        global_best_score = best_particle[1]
        global_best_model = copy.deepcopy(particles[best_particle_id].local_best_model)

        if (epoch + 1) % 10 == 0:
          print(f'Epoch {epoch + 1}/{NUM_ROUNDS}, Best Particle Loss: {global_best_score:.4f}')
          print(f"Round {epoch + 1}: Partícula {best_particle_id} está enviando os pesos.")

        server_model.load_state_dict(global_best_model)
        server_model.eval()
        total_loss, correct = 0, 0
        with torch.no_grad():
            for inputs, labels in testloader:
                inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                outputs = server_model(inputs)
                loss = criterion(outputs, labels)
                total_loss += loss.item()
                correct += (outputs.argmax(1) == labels).type(torch.float).sum().item()

        test_loss = total_loss / len(testloader)
        test_accuracy = (correct / len(testset))*100
        server_evaluate_acc.append([test_loss, test_accuracy])
        if (epoch + 1) % 10 == 0:
          print(f'Epoch {epoch + 1}/{NUM_ROUNDS}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')
          print('---')

    #write_csv("FLPSO-SGD3", architecture, server_evaluate_acc)

# Exemplo de execução
train_federated()

Epoch 10/100, Best Particle Loss: 0.0970
Round 10: Partícula 2 está enviando os pesos.
Epoch 10/100, Test Loss: 0.1815, Test Accuracy: 96.67%
---
Epoch 20/100, Best Particle Loss: 0.0124
Round 20: Partícula 2 está enviando os pesos.
Epoch 20/100, Test Loss: 0.4465, Test Accuracy: 90.00%
---
Epoch 30/100, Best Particle Loss: 0.0031
Round 30: Partícula 0 está enviando os pesos.
Epoch 30/100, Test Loss: 0.3483, Test Accuracy: 93.33%
---
Epoch 40/100, Best Particle Loss: 0.0001
Round 40: Partícula 5 está enviando os pesos.
Epoch 40/100, Test Loss: 0.6671, Test Accuracy: 80.00%
---
Epoch 50/100, Best Particle Loss: 0.0009
Round 50: Partícula 2 está enviando os pesos.
Epoch 50/100, Test Loss: 0.2544, Test Accuracy: 93.33%
---
Epoch 60/100, Best Particle Loss: 0.0038
Round 60: Partícula 9 está enviando os pesos.
Epoch 60/100, Test Loss: 0.2418, Test Accuracy: 96.67%
---
Epoch 70/100, Best Particle Loss: 0.2829
Round 70: Partícula 2 está enviando os pesos.
Epoch 70/100, Test Loss: 0.2544, Test